<a href="https://colab.research.google.com/github/skberkeley/c3.ai-COVID-19-Challenge/blob/main/Sora_Rithwik_Tyler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
from matplotlib import pyplot as plt
from scipy.stats import gamma
import numpy as np
from datetime import datetime

In [ ]:
# Returns datetime object of the given date string
# Compare dates with >, <, or ==
def timeFormat(s):
  return datetime.strptime(s, "%Y-%m-%dT%H:%M:%SZ")

# Returns float timestamp (seconds since 1 Jan 1970)
# Compare timestamps with >, <, or ==
def timestampFormat(s):
  return timeFormat(s).timestamp()

# How to convert column of date strings to these above formats:
# df["dateStringsColumn"] = df["dateStringsColumn"].apply(timeFormat)

##DON'T CHANGE:

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
import sys
sys.path.append('/content/drive/My Drive/C3.ai COVID Challenge')
import c3aidatalake

###Todo (by 11/3):
- Quantify government policy categories (LocationPolicySummary)
    - Bigger values for more restrictive policies
- Convert economic data to usable format
- Produce/visualize distributions and general statistics for data


In [ ]:
policies = c3aidatalake.fetch(
    "locationpolicysummary", 
    {
        "spec" : {
            "limit" : -1
        }
    })
policies

In [ ]:
def printColValCounts(df, colName):
    print("Column Name: " + colName)
    print(df[colName].value_counts())

In [ ]:
statePolicies = policies.drop(44)
statePolicies.columns

Generating numerical columns from statePolicies:

# Tyler
# waiveTreatmentCost', 'freeVaccine',
#      'waiverOfPriorAuthorizationRequirements', 'prescriptionRefill',
#       'premiumPaymentGracePeriod', 'marketplaceSpecialEnrollmentPeriod',
#       'section1135Waiver', 'paidSickLeaves',
#       'expandsAccesstoTelehealthServices',

In [ ]:
printColValCounts(statePolicies, "premiumPaymentGracePeriod")

In [ ]:
quantifyDicts = {}
quantifyDicts["easingOrder"] = {
    "Reopened" : 0, 
    "Proceeding with Reopening" : 1,
    "Paused" : 2, 
    "New Restrictions Imposed" : 3
}
quantifyDicts["stayAtHome"] = {
    "No Action" : 0,
    "Lifted" : 1,
    "Rolled Back to High Risk Groups" : 2,
    "New Stay at Home Order" : 3,
    "Statewide" : 4
}
quantifyDicts["waiveTreatmentCosts"]  = {
    "No Action" : 0,
    "State-Insurer Agreement" : 1,
    "State Requires" : 2
}  
quantifyDicts["freeVaccine"] = {
    "No Action" : 0,
    "State-Insurer Agreement" : 1,
    "State Requires" : 2
}
quantifyDicts["waiverofPriorAuthorizationRequirements"] = {
    "No  Action" : 0,
    "For COVID-19 Testing" : 1,
    "For COVID-19 Testing and Treatment" : 2
}
quantifyDicts["perscritpionRefill"] = {
    "No Action" : 0,
    "Expired" : 1,
    "State Requires" : 2
}
quantifyDicts["premiumPaymentGracePeriod"] = {
    "No Action" : 0,
    "Expired" : 1,
    "COVID-19 Diagnosis/Impacts Only" : 2,
    "Grace Period extended for All Individual Policies" : 3,
    "All Policies" : 4
}

quantifyDicts["marketplaceSpecialEnrollmentPeriod"] = {
    "No" : 0,
    "Ended" : 1,
    "Active" : 2
}

quantifyDicts["Section1135Waiver"] = {
    "Unapproved" : 0,
    "Approved" : 1
}
quantifyDicts["paidSickLeaves"] = {
    "No Action" : 0,
    "Proposed - March 2020" : 1,
    "Enacted" : 2
}
quantifyDicts["expandsAccesstoTelehealthServices"] = {
    "no" : 0,
    "yes" : 1
}

In [ ]:
def applyDict(val, colName, refDict):
    